# Давайте создадим AI чат бота...

## Идея

#### Давайте обсудим, для чего это нужно

1. Сильный AI может стать бесполезным, если не сможет общаться с людьми
2. Даже если не мечтать про сильный AI, умение "понимать" и осмысленно отвечать на человеческом языке может сильно помочь в автоматизации службы поддержки компании и др.
3. Изучение новой технологии

#### Цель

* Создание чат бота, поддерживающего беседу
* Изучение технологий, необходимых для этого

## Let's do this

#### Загрузим и подготовим данные

In [ ]:
!wget https://www.dropbox.com/s/tfeozvw4hfnmufu/subtitles.txt

In [ ]:
!wget https://www.dropbox.com/s/cpcfs5y6z4qqhc0/Networks_weights.npz

In [ ]:
!wget https://www.dropbox.com/s/u9873fsq6r4he28/token_id.json

In [1]:
import re
import numpy as np
import codecs
import os

PAD_ix = -1

In [2]:
def split(text):
    rep = re.findall(r">>>[А-я ]+[\.] [A-я ,;:-]+[\.\?\!\]]", text)
    return list(map(lambda x: re.sub(r">>>[А-я ]+[\.] ", "", x), rep))

In [10]:
phrase_list = []
for filename in os.listdir("data"):
    with codecs.open("data/{}".format(filename), 'r', encoding="utf-8") as f:
        text = f.read().lower()
        phrase_list.extend(split(text))
        del text

In [5]:
phrase_list[:5]

['Вот тут и живи, как хочешь.',
 'У вас ведь, маменька, уж один разговор.',
 'Что ж такое не говорить-то!',
 'Разве я виновата, маменька, что мне никто не нравится?',
 'Какой же каприз, маменька?']

In [11]:
symb = ""
for s in phrase_list:
    symb += s

Так как мы генирируем ответы посимвольно, определимся, какие символы у нас встречаются 

In [6]:
all_singhs = 'АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя0123456789.?!,;:-( )""-_*' + "''"

In [12]:
tokens = set(symb)

tokens = list(tokens)

Для обозначения начала и конца будем использовать START и END. Превратим каждое наше предложение в список символов, которые оно содержит, добавим обозначение начала и конца

In [13]:
tokens = ["START"] + tokens + ["END"]

In [14]:
len(tokens)

55

In [15]:
phrase_list = list(map(lambda x: ["START"] + list(x) + ["END"], phrase_list))

In [32]:
len(phrase_list)

4867

Создадим словарь token - id

In [16]:
token_to_id = {tok: i for i, tok in enumerate(tokens)}

id_to_token = {token_to_id[tok]: tok for tok in token_to_id.keys()}

Будем превращать все наши предложения в матрицу, имеющую форму (кол-во предложений, мак-ое кол-во символов в предложении), если остаются пустые ячейки заполним их -1

In [17]:
def as_matrix(sequences,token_to_i, max_len=None,PAX_ix=PAD_ix):
    max_len = max_len or max(map(len,sequences))
    
    matrix = np.zeros((len(sequences),max_len),dtype='int8') -1
    for i,seq in enumerate(sequences):
        row_ix = list(filter(None.__ne__, map(token_to_i.get,seq)))[:max_len]
        matrix[i,:len(row_ix)] = row_ix
    
    return matrix

Преобразуем в токены

In [18]:
print(as_matrix(phrase_list[:5], token_to_id))

[[ 0 50 31  4 20  4 23  4 20 24 20 48 24 50 24 32 20  6 17  6 20  3 31 44
  15 46 52 37 54 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1]
 [ 0 23 20 50 17  9 20 50 15 34 52 32 20 45 17 45 15 13 52  6 17 32 20 23
  48 20 31 34 24 13 20 27 17 21 41 31 50 31 27 37 54 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1]
 [ 0 44  4 31 20 48 20  4 17  6 31 15 20 13 15 20 41 31 50 31 27 24  4 52
  42  4 31 10 54 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1]
 [ 0 27 17 21 50 15 20 14 20 50 24 13 31 50 17  4 17 32 20 45 17 45 15 13
  52  6 17 32 20 44  4 31 20 45 13 15 20 13 24  6  4 31 20 13 15 20 13 27
  17 50 24  4  9 14 28 54]
 [ 0  6 17  6 31 25 20 48 15 20  6 17 33 27 24 21 32 20 45 17 45 15 13 52
   6 17 28 54 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1]]


Сохраним

In [19]:
import json

In [20]:
with open("tokens_id.json", "w") as fp:
    json.dump({"token_to_id":token_to_id, "id_to_token": id_to_token, "tokens": tokens}, fp)

In [21]:
with open("tokens_id.json") as f:
    tokens_id = json.load(f)
token_to_id = tokens_id[u'token_to_id']
id_to_token = tokens_id["id_to_token"]
tokens = tokens_id["tokens"]
tokens = tokens

# Deep learning

Создадим сеть, которая будет получать на вход вопрос и генерировать посимвольно ответ на него

In [22]:
%env THEANO_FLAGS=device=gpu5,floatX=float32
import theano
import theano.tensor as T
import lasagne
from lasagne import layers
from lasagne.objectives import categorical_crossentropy
from lasagne.updates import adam

env: THEANO_FLAGS=device=gpu5,floatX=float32


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 5: GeForce GTX 1080 (CNMeM is enabled with initial size: 22.0% of memory, cuDNN 5110)


In [23]:
input_sequence = T.matrix('token sequence','int32')
target_phonemes = T.matrix('target phonemes','int32')

In [24]:

##ENCODER
l_in = lasagne.layers.InputLayer(shape=(None, None),input_var=input_sequence)
l_mask = lasagne.layers.InputLayer(shape=(None, None),input_var=T.neq(input_sequence,-1))
l_emb = lasagne.layers.EmbeddingLayer(l_in, len(tokens), 40)
l_rnn = lasagne.layers.LSTMLayer(l_emb,256,mask_input=l_mask)
l_rnn = lasagne.layers.LSTMLayer(l_rnn,256,only_return_final=True,mask_input=l_mask)

##DECODER
transc_in = lasagne.layers.InputLayer(shape=(None, None),input_var=target_phonemes)
transc_mask = lasagne.layers.InputLayer(shape=(None, None),input_var=T.neq(target_phonemes,-1))
transc_emb = lasagne.layers.EmbeddingLayer(transc_in, len(tokens), 50)
transc_rnn = lasagne.layers.LSTMLayer(transc_emb,256,hid_init=l_rnn,mask_input=transc_mask)
transc_rnn = lasagne.layers.LSTMLayer(transc_rnn,256,hid_init=l_rnn,mask_input=transc_mask)


#flatten batch and time to be compatible with feedforward layers (will un-flatten later)
transc_rnn_flat = lasagne.layers.reshape(transc_rnn, (-1,transc_rnn.output_shape[-1]))

l_out = lasagne.layers.DenseLayer(transc_rnn_flat,len(tokens),nonlinearity=lasagne.nonlinearities.softmax)



In [25]:
weights = lasagne.layers.get_all_params(l_out, trainable=True)
print(weights)

[W, W, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W, b]


In [26]:
network_output = lasagne.layers.get_output(l_out)
network_output = network_output.reshape([target_phonemes.shape[0], target_phonemes.shape[1], -1])

In [27]:
predictions_flat = network_output[:,:-1,:].reshape([-1, len(tokens)])
targets = target_phonemes[:,1:].ravel()

mask = T.nonzero(T.neq(targets, -1))

loss = categorical_crossentropy(predictions_flat[mask], targets[mask]).mean()
updates = adam(loss, weights)

Компилируем

In [28]:

#training
train = theano.function([input_sequence, target_phonemes], loss, updates=updates, allow_input_downcast=True)

#computing loss without training
compute_cost = theano.function([input_sequence, target_phonemes], loss, allow_input_downcast=True)


## Создадим генератор ответов

In [29]:
#compile the function that computes probabilities for next token given previous text.

#reshape back into original shape
network_output = network_output.reshape((target_phonemes.shape[0],target_phonemes.shape[1],len(tokens)))
#predictions for next tokens (after sequence end)
last_word_probas = network_output[:,-1]
probs = theano.function([input_sequence,target_phonemes],last_word_probas,allow_input_downcast=True)


In [30]:
def generate_answer(question,answer_prefix = ("START",),t=1,sample=True, max_len=500):
    
    answer = list(answer_prefix)
    for _ in range(max_len):
        
        next_let_probs = probs(as_matrix([question],token_to_id),as_matrix([answer],token_to_id) ).ravel()
        next_let_probs = next_let_probs**t / np.sum(next_let_probs**t)

        if sample:
            next_letter = np.random.choice(tokens,p=next_let_probs) 
        else:
            next_letter = tokens[np.argmax(next_let_probs)]
        
        answer.append(next_letter)

        if next_letter=="END":
            break
    return "".join(answer[1:-1])

In [36]:
print(generate_answer("Привет"))

лаSTARTнsSTARTфд?iхя г льсжduъньйчцфiьpс


## Загрузим предобученые веса, если есть

In [ ]:
with np.load("Networks_weights.npz", encoding="bytes") as weights_file:
    lasagne.layers.set_all_param_values(l_out, weights_file["arr_0"])

### Тренируем

In [37]:
def iterate_minibatches(source, batchsize, shuffle=True):
    source = np.array(source)
    if shuffle:
        indices = np.arange(len(source)-1)
        np.random.shuffle(indices)
    for start_idx in range(0, len(source) -1 - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = np.array(slice(start_idx, start_idx + batchsize))
        yield as_matrix(source[excerpt], token_to_id), as_matrix(source[excerpt+1],token_to_id)

In [38]:
import config

In [39]:
telegram_token = config.telegram_token
telegram_chat_id = config.telegram_chat_id# id чата, в который будут отсылаться результаты в процессе обучения, так как она учится долго
dropbox_token = config.dropbox_token# Свежие веса загружаются на в облако, чтобы не потерять

In [40]:
from telebot import TeleBot
bot = TeleBot(telegram_token)

In [ ]:
import dropbox

client = dropbox.client.DropboxClient(dropbox_token)

In [41]:
train_message = "Epoch {epoch}:\nTime {time_:.2f} hours\nAverage loss: {avg_los:.5f}\nExample sample=True: {sample_0}"
train_error_messages = "Error on epoch when we train model: {}\n\t{}\nBatch:\n{}"

In [43]:
from time import time
num_epoch = 100
batch_per_epoch = 1000
batch_size = 100
bot.send_message(145718567, "Learning is begin!")
for epoch in range(num_epoch):
    try:
        
        st = time()
        avg_cost = 0
        for batch_num, batch in enumerate(iterate_minibatches(phrase_list, batch_size)):
            try:
                avg_cost += train(batch[0], batch[1])
            except Exception as er:
                bot.send_message(telegram_chat_id, train_error_messages.format(epoch+1, er, batch))
                break
                
            if batch_num+1 == batch_per_epoch:
                break
                
        weights_file = "Networks_weights.npz"
        np.savez(weights_file, layers.get_all_param_values(l_out))
#         try:
#             with open(weights_file, 'rb') as weights_file_dr:
#                 response = client.put_file('/' + weights_file, weights_file_dr)

#         except Exception as er:
#             bot.send_message(telegram_chat_id, "Error of download to Dropbox: {}".format(e))

        bot.send_message(145718567, train_message.format(time_=(time()-st)/3600,
                                                         epoch=epoch+1,
                                                         avg_los=avg_cost/batch_per_epoch,
                                                         sample_0=generate_answer("Здравствуй")))
    
    except Exception as er:
        bot.send_message(telegram_chat_id, "Error on epoch: {}\n\t{}".format(epoch+1, er))
        break
bot.send_message(telegram_chat_id, "Learning is end!")

KeyboardInterrupt: 

In [49]:
generate_answer("Как тебя зовут?", t=1)

'беспеться; нешенька упержи маменьку не сославне тятеновену.'